In [ ]:
# default_exp sewingbot
# hide
_FNAME='sewingbot'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os
TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')

In [ ]:
#export
from pprint import pprint
import json
import logging
import time
import datetime

import discord
from discord.ext import tasks, commands

import sewing
logger = sewing.start_log()
import sewing.db as db
from sewing.notify import Retriever, send_content

from sewing.config import configs

In [ ]:
#export 

description = '''Summarize threads and send as an email'''
command_prefix='||'
bot = commands.Bot(command_prefix=command_prefix, description=description)

In [ ]:
#export
async def guilds():
    guilds = db.guilds()
    guild_objects = []
    for guild_d in guilds:
        guild_id = guild_d['guild']['id']
        guild = await bot.fetch_guild(guild_id)
        guild_objects.append(guild)
    return guild_objects

async def get_all_channel_content(channels):
    retriever = Retriever()    
    channels_content = {}
    for channel_d in channels:
        channel_id = channel_d['channel']['id']
        channel_obj = await bot.fetch_channel(int(channel_id))
        #{thread: [messages]}
        messages = await retriever.get_all_messages_in_channel(channel_obj)
        #print(json.dumps(messages))
        channels_content[channel_obj] = messages
    return channels_content 
    
async def get_guild_content(guild):
    guild_content = {}
    channels = db.summarized_channels(guild, names_only=False)
    channels_content = await get_all_channel_content(channels)
    guild_content[guild] = channels_content
    return guild_content

async def sew_threads(guild):
    logger.info("sewing threads")
    guild_content = await get_guild_content(guild)
    logger.info("{}".format(guild_content)
    resps = send_content(guild_content)
    logger.info('{}'.format([resp.status_code for resp in resps]))
    
@tasks.loop(minutes=1)
async def check_if_time_to_send():
    now = time.time()
    all_guilds = await guilds()

    for guild in all_guilds:
        last_report = db.last_report(guild)
        logger.info("The last report for {guild} was {lr}".format(guild=guild.name, lr=last_report))
        send = False
        if last_report is None:
            send = True
        else:
            seconds_since_report = now-last_report
            seconds_per_report = datetime.timedelta(**configs['frequency']).total_seconds()
            logger.info("{ssr} and the report needs me to wait {spr}".format(ssr=seconds_since_report, spr=seconds_per_report))
            if seconds_since_report > seconds_per_report:
                send = True
                
        logger.info("sending report for {guild}: {send}".format(guild=guild.name, send=send))
        if send:
            await sew_threads(guild)
            db.add_report(guild, ts=now)                

@bot.event
async def on_ready():
    print(f'Logged in as {bot.user} (ID: {bot.user.id})')
    print('------')
    db.ensure_unique_nodes()
    for guild in bot.guilds:
        db.ensure_guild(guild)
    check_if_time_to_send.start()

## Register Users

In [ ]:
#export

@bot.command(description="Registers you to receive email updates")
async def register(ctx):
    guild = ctx.guild
    author = ctx.author
    
    if guild is None:
        await author.send("I can't tell which guild you're registering for.  `{}register` from a guild channel".format(command_prefix))
        return None
    print(guild, ctx.author.mutual_guilds)
    resp = db.register_user(author, guild)
    record = db.get_email(author)['email']
    email_text = 'We have no email address for you'
    
    if record:
        email_text = 'Your email is {}'.format(record)
    await author.send("You're signed up. {email_text}.  `{cp}email me@email.com` to change your email address".format(
            email_text=email_text, cp=command_prefix))

@bot.command(description="You stop getting email updates")
async def unregister(ctx):
    guild = ctx.guild
    author = ctx.author
    
    if guild is None:
        await author.send("I can't tell which guild you're unregistering for.  `{cp}unregister` from a guild channel".format(cp=command_prefix))
        return None    
    resp = db.unregister_user(author, guild)
    await ctx.send("Was it something I said?  (You'll stop getting summary emails from me).")
    
@bot.command(description="Add your email address")
async def email(ctx, address:str):
    author = ctx.author
    resp = db.set_email(author, address)
    await author.send("Email address recorded")
                      

## Add channels to be summarized

In [ ]:
#export

@bot.command(description="START or STOP threads on this channel from being summarized")
async def summarize(ctx, cmd:str=''):
    guild = ctx.guild
    channel = ctx.channel
    
    cmd = cmd.upper()
    
    if cmd not in ('START', 'STOP'):
        await ctx.send("Usage: `{cp}summarize start` or `{cp}summarize stop`".format(cp=command_prefix))
    
    if cmd=='START':
        resp = db.add_channel(channel, guild)
        
    elif cmd=='STOP':
        resp = db.remove_channel(channel, guild)

    all_channels = db.summarized_channels(guild, names_only=True)
    if all_channels:
        msg = "Channels being summarized are: {}".format(','.join(all_channels))
    else:
        msg = "No channels are being summarized"
    await ctx.send(msg)

## Let the users suggest features

In [ ]:
#export

@bot.command()
async def suggest(ctx, *args):
    user = ctx.author
    db.add_suggestion(user, *args)
    await ctx.send("Thanks for the suggestion.  Its been passed along to the bot's developer")

## Help message

In [ ]:
#export

help_text='''
You can `{cp}register` for email updates of important threads.  
When you do, I'll DM you so you can set your email address `{cp}email my@address.com`.
`{cp}unregister` to stop getting email updates.
You can also `{cp}suggest your good ideas` to the devs of this bot.
Finally you can `{cp}summarize start` or `{cp}summarize stop` to add/remove channels from the summary.  
Be careful though, this is not done user-by-user yet so you'll include that channel for everyone.
    '''.format(cp=command_prefix)
help_text

"\nYou can `||register` for email updates of important threads.  \nWhen you do, I'll DM you so you can set your email address `||email my@address.com`.\n`||unregister` to stop getting email updates.\nYou can also `||suggest your good ideas` to the devs of this bot.\nFinally you can `||summarize start` or `||summarize stop` to add/remove channels from the summary.  \nBe careful though, this is not done user-by-user yet so you'll include that channel for everyone.\n    "

In [ ]:
#export
@bot.command("how")
async def helpme(ctx):

    await ctx.send(help_text)
    

CommandRegistrationError: The command how is already an existing command or alias.

In [ ]:
#export
import os
if sewing.is_main(globals()):
    logger.setLevel(logging.INFO)

    token = os.environ['DISCORD_TOKEN']
    #client.run(token)
    bot.run(token)

In [ ]:
notebook2script(_nbpath)

Converted sewingbot.ipynb.
